In [53]:
from bs4 import BeautifulSoup
import requests
import re
from selenium import webdriver

In [156]:
browser = webdriver.Chrome("webdriver/chromedriver.exe")

c:\users\work\anaconda3\envs\kia\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  """Entry point for launching an IPython kernel.


In [130]:
main_url = 'https://edition.cnn.com'
# r = requests.get(url)
browser.implicitly_wait(5)
browser.get(main_url)
browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [131]:
main_html = browser.page_source
main_soup = BeautifulSoup(main_html)

In [175]:
print("[INFO] Scraping categories and sub-categories ...")
list_cat_tag = main_soup.find_all("a",class_="sc-fjdhpX sc-chPdSV hnOkcW")
dict_cat_profile = {}
for i, cat in enumerate(list_cat_tag):
    if cat.text in dict_cat_profile.keys():
        continue
    cat_profile = {}
    cat_profile["url"] = cat["href"]
    
    subcat_tags = cat.find_next_sibling('ul')
    subcats = subcat_tags.find_all("a")
    list_subcat_href = {}
    for subcat in subcats:
        list_subcat_href[subcat.text]=subcat["href"]
        
    cat_profile["subcat"] = list_subcat_href
    
    dict_cat_profile[cat.text] = cat_profile

print(f"[INFO] found {len(dict_cat_profile)} categories: {[k for k in dict_cat_profile.keys()]}")

[INFO] Scraping categories and sub-categories ...
[INFO] found 13 categories: ['World', 'US Politics', 'Business', 'Health', 'Entertainment', 'Tech', 'Style', 'Travel', 'Sports', 'Videos', 'Features', 'Weather', 'More']


In [139]:
dict_cat_profile

{'World': {'url': '/world',
  'subcat': {'Africa': '/africa',
   'Americas': '/americas',
   'Asia': '/asia',
   'Australia': '/australia',
   'China': '/china',
   'Europe': '/europe',
   'India': '/india',
   'Middle East': '/middle-east',
   'United Kingdom': '/uk'}},
 'US Politics': {'url': '/politics',
  'subcat': {'The Biden Presidency': '/specials/politics/joe-biden-news',
   'Facts First': '/specials/politics/fact-check-politics',
   'US Elections': '/election/2022'}},
 'Business': {'url': '/business',
  'subcat': {'Markets': 'https://money.cnn.com/data/markets/',
   'Tech': '/business/tech',
   'Media': '/business/media',
   'Success': '/business/success',
   'Perspectives': '/business/perspectives',
   'Videos': '/business/videos'}},
 'Health': {'url': '/health',
  'subcat': {'Life, But Better': '/specials/health/life-but-better',
   'Fitness': '/specials/health/fitness-life-but-better',
   'Food': '/specials/health/food-life-but-better',
   'Sleep': '/specials/health/sleep-l

In [140]:
list_article_url = []
print("[INFO] Scraping article urls ...")
for k, v in dict_cat_profile.items():
    cat_name = k
    url = v["url"]
    # r = requests.get(url)
    browser.implicitly_wait(5)
    browser.get(main_url+url)
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    html = browser.page_source
    soup = BeautifulSoup(html)
    article_div = soup.find_all(class_="cd__content")
    print(f"[INFO] Scraping from {main_url+url}: found {len(article_div)} articles")
    list_article_url.extend([a.find("a")["href"] for a in article_div])
    



https://edition.cnn.com/world: found 112 articles
https://edition.cnn.com/politics: found 96 articles
https://edition.cnn.com/business: found 56 articles
https://edition.cnn.com/health: found 83 articles
https://edition.cnn.com/entertainment: found 40 articles


WebDriverException: Message: unknown error: cannot determine loading status
from unknown error: unexpected command response
  (Session info: chrome=103.0.5060.114)
Stacktrace:
Backtrace:
	Ordinal0 [0x00B96463+2188387]
	Ordinal0 [0x00B2E461+1762401]
	Ordinal0 [0x00A43D78+802168]
	Ordinal0 [0x00A37210+750096]
	Ordinal0 [0x00A3675A+747354]
	Ordinal0 [0x00A35D3F+744767]
	Ordinal0 [0x00A34C28+740392]
	Ordinal0 [0x00A35228+741928]
	Ordinal0 [0x00A3EF2F+782127]
	Ordinal0 [0x00A49FBB+827323]
	Ordinal0 [0x00A4D310+840464]
	Ordinal0 [0x00A354F6+742646]
	Ordinal0 [0x00A49BF3+826355]
	Ordinal0 [0x00A9CF6D+1167213]
	Ordinal0 [0x00A8C5F6+1099254]
	Ordinal0 [0x00A66BE0+945120]
	Ordinal0 [0x00A67AD6+948950]
	GetHandleVerifier [0x00E371F2+2712546]
	GetHandleVerifier [0x00E2886D+2652765]
	GetHandleVerifier [0x00C2002A+520730]
	GetHandleVerifier [0x00C1EE06+516086]
	Ordinal0 [0x00B3468B+1787531]
	Ordinal0 [0x00B38E88+1805960]
	Ordinal0 [0x00B38F75+1806197]
	Ordinal0 [0x00B41DF1+1842673]
	BaseThreadInitTh

In [147]:
def validate_article_url(url):
    valid = True
    if "http" in url:
        valid = False
        
    if "/video/" in url:
        valid = False
        
    if "/videos/" in url:
        valid = False
        
    return valid

validate_article_url('/videos/politics/2022/07/12/jake-tapper-john-bolton-debate-january-6-coup-attempt-sot-lead-vpx.cnn/video/playlists/this-week-in-politics/')

False

In [153]:
list_valid_article_url = [url for url in list_article_url if validate_article_url(url)]

In [192]:
valid_article_url = list_valid_article_url[4]

In [193]:
print(main_url+valid_article_url)
r = requests.get(main_url+valid_article_url)
# browser.implicitly_wait(5)
# browser.get(main_url)
# browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
main_soup = BeautifulSoup(r.content, "html.parser")

https://edition.cnn.com/2022/07/13/europe/ukraine-russia-next-stages-donetsk-intl/index.html


In [202]:
#title
main_soup.find("h1").text

'Two exhausted armies are battling for eastern Ukraine. Can either of them strike a decisive blow?'

In [194]:
#metadata
metadata_div = main_soup.find(class_="metadata")
print(metadata_div)
author_div = metadata_div.find(class_="metadata__byline__author")
author_name = author_div.text
author_url = author_div.find("a")
update_time = metadata_div.find(class_="update-time").text
author_name, author_url

<div class="metadata"><div class="metadata__info js-byline-images" data-bundle="byline"><p class="metadata__byline"><span class="metadata__byline__author">Analysis by <a href="https://twitter.com/robpicheta" target="_blank">Rob Picheta</a>, CNN</span></p><p class="metadata__show"></p><p class="update-time">Updated 2141 GMT (0541 HKT) July 13, 2022 <span class="video__source top_source" id="js-pagetop_video_source"></span></p></div></div>


('Analysis by Rob Picheta, CNN',
 <a href="https://twitter.com/robpicheta" target="_blank">Rob Picheta</a>)

In [195]:
list_related_article_urls = []
for p in main_soup.find_all(class_="zn-body__paragraph"):
    a_tags = p.find_all("a")
    for a in a_tags:
        list_related_article_urls.append(a['href'])
    print(p.text)

 (CNN)When Vladimir Putin refocused his war in Ukraine on the country's east three months ago, he did so bruised by the failures of his initial lunge towards Kyiv and desperate for a face-saving success.
After a slow and bloody march through Luhansk was finalized with the capture of the city of Lysychansk, the Russian President might consider himself halfway there. 
But the war has arrived at another crossroads and fighters on both sides are steeling themselves for a third act of fighting that could tip the balance of the conflict. 
"It's a very attritional struggle," said Justin Bronk, a senior research fellow for Airpower and Technology at the Royal United Services Institute (RUSI), describing the tone of the war after three months of fighting in Donbas.
"It's a struggle between two armies, both of whom have taken huge losses and are very close to exhaustion."
Putin's next move is anticipated to be a drive into Donetsk, which if captured would fulfill the Kremlin's primary objective:

In [196]:
list_related_article_urls

['http://www.cnn.com/2022/04/15/europe/donbas-region-ukraine-war-russia-explainer-intl/index.html',
 'https://www.cnn.com/2022/07/03/europe/russia-ukraine-luhansk-lysychansk-intl/index.html',
 'https://www.cnn.com/2022/04/15/europe/donbas-region-ukraine-war-russia-explainer-intl/index.html',
 'http://www.cnn.com/2022/06/27/europe/russia-ukraine-war-tide-turning-lister-intl-hnk/index.html',
 'https://www.understandingwar.org/backgrounder/russian-offensive-campaign-assessment-july-7',
 'https://twitter.com/DefenceHQ/status/1530055732271452161',
 'https://www.cnn.com/europe/live-news/russia-ukraine-war-news-07-06-22/h_7c51806872728ce6ee905df2b99fe1e3',
 'https://www.cnn.com/2022/06/03/politics/ukraine-100-days-western-allies-regular-meetings-potential-ceasefire/index.html']